<a href="https://colab.research.google.com/github/harish-babu/stash-ai-ml-models/blob/NLP-Automatic-Ticket-Classification/Automatic_Ticket_Classification_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem Statement

You need to build a model that is able to classify customer complaints based on the products/services. By doing so, you can segregate these tickets into their relevant categories and, therefore, help in the quick resolution of the issue.

You will be doing topic modelling on the <b>.json</b> data provided by the company. Since this data is not labelled, you need to apply NMF to analyse patterns and classify tickets into the following five clusters based on their products/services:

* Credit card / Prepaid card

* Bank account services

* Theft/Dispute reporting

* Mortgages/loans

* Others


With the help of topic modelling, you will be able to map each ticket onto its respective department/category. You can then use this data to train any supervised model such as logistic regression, decision tree or random forest. Using this trained model, you can classify any new customer complaint support ticket into its relevant department.

## Pipelines that needs to be performed:

You need to perform the following eight major tasks to complete the assignment:

1.  Data loading

2. Text preprocessing

3. Exploratory data analysis (EDA)

4. Feature extraction

5. Topic modelling

6. Model building using supervised learning

7. Model training and evaluation

8. Model inference

## Importing the necessary libraries

In [9]:
# Install Swifter Library to efficiently apply any function to a pandas dataframe or series in the fastest available manner.
!pip install swifter

In [10]:
import json
import numpy as np
import pandas as pd
import re, nltk, spacy, string
import en_core_web_sm
nlp = en_core_web_sm.load()
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from plotly.offline import plot
import plotly.graph_objects as go
import plotly.express as px

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pprint import pprint

import swifter


## Loading the data

The data is in JSON format and we need to convert it to a dataframe.

In [11]:
# Source Path of thecomplaints-data-ticket-classification.zip file.
# If it is gdrive, then it will point to that.  If it is in local, it will show that.
# Make sure you mount the google drive in /content/gdrive

zip_file_src_path = '/content/gdrive/MyDrive/NLP-Automatic-Ticket-Classification'   # For running from Google Drive
#zip_file_src_path = './'                                                           # For local running of notebook

In [12]:
# Define a base path.  It can be changed when moving from colab to local
base_path = "/content/data"                                                           # This is for local data on the GPU m/c
# base_path = "."                                                                     # This is for local


In [13]:
# If you are using the data by mounting the google drive, use the following :
from google.colab import drive
import pathlib
from pathlib import Path
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [14]:
file_name = 'complaints-data-ticket-classification.zip'

In [15]:
# Now let us unzip the file to local drive

from zipfile import ZipFile
with ZipFile(zip_file_src_path + '/' + file_name,'r') as zip:
  zip.extractall(base_path)

In [16]:
# Opening JSON file

f = open(base_path + '/complaints-2021-05-14_08_16.json')

# returns JSON object as
# a dictionary
data = json.load(f)
df=pd.json_normalize(data)

## Data preparation

In [17]:
# Inspect the dataframe to understand the given data.

df.head()

,_index,_type,_id,_score,_source.tags,_source.zip_code,_source.complaint_id,_source.issue,_source.date_received,_source.state,...,_source.company_response,_source.company,_source.submitted_via,_source.date_sent_to_company,_source.company_public_response,_source.sub_product,_source.timely,_source.complaint_what_happened,_source.sub_issue,_source.consumer_consent_provided
0,complaint-public-v2,complaint,3211475,0.0,None,90301,3211475,Attempts to collect debt not owed,2019-04-13T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-13T12:00:00-05:00,None,Credit card debt,Yes,,Debt is not yours,Consent not provided
1,complaint-public-v2,complaint,3229299,0.0,Servicemember,319XX,3229299,Written notification about debt,2019-05-01T12:00:00-05:00,GA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-05-01T12:00:00-05:00,None,Credit card debt,Yes,Good morning my name is XXXX XXXX and I apprec...,Didn't receive enough information to verify debt,Consent provided
2,complaint-public-v2,complaint,3199379,0.0,None,77069,3199379,"Other features, terms, or problems",2019-04-02T12:00:00-05:00,TX,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-02T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Problem with rewards from credit card,Consent provided
3,complaint-public-v2,complaint,2673060,0.0,None,48066,2673060,Trouble during payment process,2017-09-13T12:00:00-05:00,MI,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2017-09-14T12:00:00-05:00,None,Conventional home mortgage,Yes,,None,Consent not provided
4,complaint-public-v2,complaint,3203545,0.0,None,10473,3203545,Fees or interest,2019-04-05T12:00:00-05:00,NY,...,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2019-04-05T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,,Charged too much interest,N/A


In [18]:
#print the column names
df.columns

Index(['_index', '_type', '_id', '_score', '_source.tags', '_source.zip_code',
       '_source.complaint_id', '_source.issue', '_source.date_received',
       '_source.state', '_source.consumer_disputed', '_source.product',
       '_source.company_response', '_source.company', '_source.submitted_via',
       '_source.date_sent_to_company', '_source.company_public_response',
       '_source.sub_product', '_source.timely',
       '_source.complaint_what_happened', '_source.sub_issue',
       '_source.consumer_consent_provided'],
      dtype='object')

### Identifying Key Columns
By looking at the data and columns, we can infer that the key columns that provide information are '_source.complaint_what_happened', '_source.product', '_source.sub_product'.  '_source.complaint_what_happened' has details about the customer complaint.  The other two columns provide information about the product & subproduct that the customer was using.*italicised text*

#### Select Key Columns & Assign Readable Column Names
The column names are not very reader friendly.  We can rename them to be easier to read and understand.

In [19]:
# Select Key Columns
df = df[['_source.complaint_what_happened','_source.product','_source.sub_product',]]
df.head()


,_source.complaint_what_happened,_source.product,_source.sub_product
0,,Debt collection,Credit card debt
1,Good morning my name is XXXX XXXX and I apprec...,Debt collection,Credit card debt
2,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Credit card or prepaid card,General-purpose credit card or charge card
3,,Mortgage,Conventional home mortgage
4,,Credit card or prepaid card,General-purpose credit card or charge card


In [20]:
# Assign new column names
column_dict = {
    '_source.complaint_what_happened': 'complaint_text',
    '_source.product': 'category',
    '_source.sub_product': 'sub_category'
}
df.rename(columns=column_dict, inplace=True)
df.head()

,complaint_text,category,sub_category
0,,Debt collection,Credit card debt
1,Good morning my name is XXXX XXXX and I apprec...,Debt collection,Credit card debt
2,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Credit card or prepaid card,General-purpose credit card or charge card
3,,Mortgage,Conventional home mortgage
4,,Credit card or prepaid card,General-purpose credit card or charge card


Like we said earlier, the category and sub_category fields will not be used directly for topic modelling.  However, they will be used later during interpretation of the groups create after NMF step.

In [21]:
# Assign nan in place of blanks in the complaints column
df.complaint_text.isnull().sum()

0

In [22]:
count_empty = (df['complaint_text'] == '').sum()
count_empty

57241

In [23]:
# Remove all rows where complaint_text is empty

df = df[df['complaint_text'].ne('')]

## Prepare the text for topic modeling

Once you have removed all the blank complaints, you need to:

* Make the text lowercase
* Remove text in square brackets
* Remove punctuation
* Remove words containing numbers


Once you have done these cleaning operations you need to perform the following:
* Lemmatize the texts
* Extract the POS tags of the lemmatized text and remove all the words which have tags other than NN[tag == "NN"].


In [24]:
# Write your function here to clean the text and remove all the unnecessary elements.

# Clean various unnecessary elements from the text
def clean_data(text):
    text = text.lower()                                                           # text to lowercase
    text = re.sub(r'\s\{\$\S*', '',text)                                          # Remove text within curly braces
    text = re.sub(r'^\[[\w\s]\]+$',' ',text)                                      # Remove text in square brackets
    text = re.sub(r'\n', '', text)                                                # Remove line breaks
    text = re.sub(r'\(\w*\)', '', text)                                           # Remove text within braces
    text = re.sub(r'(\W\s)|(\W$)|(\W\d*)', ' ',text)                              # Remove punctuation
    text = re.sub(r'x+((/xx)*/\d*\s*)|x*', '',text)                               # Remove date
    text = re.sub(r'\d+\s', '', text)                                             # Remove other numerical values
    text = re.sub(r' +', ' ',text)                                                # Remove unnecessary white spaces
    return text

In [25]:
# Write your function to Lemmatize the texts
def lemmatization(text):
    return ' '.join([token.lemma_ for token in nlp(text) if token.text.lower() not in nlp.Defaults.stop_words])

In [26]:
# Create a dataframe('df_clean') that will have only the complaints and the lemmatized complaints
df_clean = pd.DataFrame(df['complaint_text'].swifter.apply(clean_data))

Pandas Apply:   0%|          | 0/21072 [00:00<?, ?it/s]

In [ ]:
df_clean['complaint_text_lemmatized'] = df_clean['complaint_text'].swifter.apply(lemmatization)

Pandas Apply:   0%|          | 0/21072 [00:00<?, ?it/s]

In [32]:
df_clean

,complaint_text,complaint_text_lemmatized
1,good morning my name is and i appreciate it if...,good morning appreciate help stop chase bank c...
2,i upgraded my card in and was told by the agen...,upgrade card tell agent upgrade anniversary da...
10,chase card was reported on however fraudulent ...,chase card report fraudulent application submi...
11,on while trying to book a ticket i came across...,try book ticket come offer apply ticket apply ...
14,my grand son give me check for i deposit it in...,grand son check deposit chase account fund cle...
...,...,...
78303,after being a chase card customer for well ove...,chase card customer decade offer multiple soli...
78309,on wednesday i called chas my visa credit card...,wednesday call chas visa credit card provider ...
78310,i am not familiar with pay and did not underst...,familiar pay understand great risk provide con...
78311,i have had flawless credit for yrs i ve had ch...,flawless credit yrs ve chase credit card chase...


In [37]:
#Write your function to extract the POS tags

def pos_tag(text):
    # write your code here
    return ' '.join([token.lemma_ for token in nlp(text) if token.tag_ == 'NN'])


df_clean["complaint_POS_removed"] =  df_clean['complaint_text_lemmatized'].swifter.apply(lambda x: pos_tag(x))


Pandas Apply:   0%|          | 0/21072 [00:00<?, ?it/s]

In [38]:
#The clean dataframe should now contain the raw complaint, lemmatized complaint and the complaint after removing POS tags.
df_clean

,complaint_text,complaint_text_lemmatized,complaint_POS_removed
1,good morning my name is and i appreciate it if...,good morning appreciate help stop chase bank c...,morning appreciate help bank service write cha...
2,i upgraded my card in and was told by the agen...,upgrade card tell agent upgrade anniversary da...,card agent upgrade anniversary date change age...
10,chase card was reported on however fraudulent ...,chase card report fraudulent application submi...,chase card report application submit identity ...
11,on while trying to book a ticket i came across...,try book ticket come offer apply ticket apply ...,book ticket come offer ticket reward card info...
14,my grand son give me check for i deposit it in...,grand son check deposit chase account fund cle...,son deposit chase account fund bank account pa...
...,...,...,...
78303,after being a chase card customer for well ove...,chase card customer decade offer multiple soli...,chase card customer decade solicitation credit...
78309,on wednesday i called chas my visa credit card...,wednesday call chas visa credit card provider ...,visa credit card provider ask claim purchase p...
78310,i am not familiar with pay and did not underst...,familiar pay understand great risk provide con...,pay risk consumer year trust mobile banking te...
78311,i have had flawless credit for yrs i ve had ch...,flawless credit yrs ve chase credit card chase...,credit yrs credit card chase freedom problem t...


## Exploratory data analysis to get familiar with the data.

Write the code in this task to perform the following:

*   Visualise the data according to the 'Complaint' character length
*   Using a word cloud find the top 40 words by frequency among all the articles after processing the text
*   Find the top unigrams,bigrams and trigrams by frequency among all the complaints after processing the text. ‘




In [49]:
# Write your code here to visualise the data according to the 'Complaint' character length
doc_lengths = pd.DataFrame()

doc_lengths['complaint_text'] = [len(d) for d in df_clean['complaint_text']]
doc_lengths['complaint_text_lemmatized'] = [len(d) for d in df_clean['complaint_text_lemmatized']]
doc_lengths['complaint_POS_removed'] = [len(d) for d in df_clean['complaint_POS_removed']]

max_length = doc_lengths.max().max()

In [56]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=doc_lengths['complaint_text'], name='Complaint'))
fig.add_trace(go.Histogram(x=doc_lengths['complaint_text_lemmatized'], name='Complaint Lemmatized'))
fig.add_trace(go.Histogram(x=doc_lengths['complaint_POS_removed'], name='Complaint POS Removed'))
fig.update_layout(barmode='overlay', title='Complaint Character Length', xaxis_title='Character Length', yaxis_title='Count')
fig.update_traces(opacity=0.75)
fig.update_layout(xaxis_range=[0,max_length])
fig.show()

We can see that there are a lot of reviews with less than 5K words in them and then a long tail of reviews with the longest one having 29872 words.

#### Find the top 40 words by frequency among all the articles after processing the text.

In [ ]:
# Using a word cloud find the top 40 words by frequency among all the articles
# after processing the text


In [ ]:
# Removing -PRON- from the text corpus
df_clean['Complaint_clean'] = df_clean['complaint_POS_removed'].str.replace('-PRON-', '')

#### Find the top unigrams,bigrams and trigrams by frequency among all the complaints after processing the text.

In [ ]:
#Write your code here to find the top 30 unigram frequency among the complaints in the cleaned datafram(df_clean).


In [ ]:
#Print the top 10 words in the unigram frequency


In [ ]:
#Write your code here to find the top 30 bigram frequency among the complaints in the cleaned datafram(df_clean).


In [ ]:
#Print the top 10 words in the bigram frequency

In [ ]:
#Write your code here to find the top 30 trigram frequency among the complaints in the cleaned datafram(df_clean).


In [ ]:
#Print the top 10 words in the trigram frequency

## The personal details of customer has been masked in the dataset with xxxx. Let's remove the masked text as this will be of no use for our analysis

In [ ]:
df_clean['Complaint_clean'] = df_clean['Complaint_clean'].str.replace('xxxx','')

In [ ]:
#All masked texts has been removed
df_clean

## Feature Extraction
Convert the raw texts to a matrix of TF-IDF features

**max_df** is used for removing terms that appear too frequently, also known as "corpus-specific stop words"
max_df = 0.95 means "ignore terms that appear in more than 95% of the complaints"

**min_df** is used for removing terms that appear too infrequently
min_df = 2 means "ignore terms that appear in less than 2 complaints"

In [ ]:
#Write your code here to initialise the TfidfVectorizer



#### Create a document term matrix using fit_transform

The contents of a document term matrix are tuples of (complaint_id,token_id) tf-idf score:
The tuples that are not there have a tf-idf score of 0

In [ ]:
#Write your code here to create the Document Term Matrix by transforming the complaints column present in df_clean.


## Topic Modelling using NMF

Non-Negative Matrix Factorization (NMF) is an unsupervised technique so there are no labeling of topics that the model will be trained on. The way it works is that, NMF decomposes (or factorizes) high-dimensional vectors into a lower-dimensional representation. These lower-dimensional vectors are non-negative which also means their coefficients are non-negative.

In this task you have to perform the following:

* Find the best number of clusters
* Apply the best number to create word clusters
* Inspect & validate the correction of each cluster wrt the complaints
* Correct the labels if needed
* Map the clusters to topics/cluster names

In [ ]:
from sklearn.decomposition import NMF

## Manual Topic Modeling
You need to do take the trial & error approach to find the best num of topics for your NMF model.

The only parameter that is required is the number of components i.e. the number of topics we want. This is the most crucial step in the whole topic modeling process and will greatly affect how good your final topics are.

In [ ]:
#Load your nmf_model with the n_components i.e 5
num_topics = #write the value you want to test out

#keep the random_state =40
nmf_model = #write your code here

In [ ]:
nmf_model.fit(dtm)
len(tfidf.get_feature_names())

In [ ]:
#Print the Top15 words for each of the topics


In [ ]:
#Create the best topic for each complaint in terms of integer value 0,1,2,3 & 4



In [ ]:
#Assign the best topic to each of the cmplaints in Topic Column

df_clean['Topic'] = #write your code to assign topics to each rows.

In [ ]:
df_clean.head()

In [ ]:
#Print the first 5 Complaint for each of the Topics
df_clean=df_clean.groupby('Topic').head(5)
df_clean.sort_values('Topic')

#### After evaluating the mapping, if the topics assigned are correct then assign these names to the relevant topic:
* Bank Account services
* Credit card or prepaid card
* Theft/Dispute Reporting
* Mortgage/Loan
* Others

In [ ]:
#Create the dictionary of Topic names and Topics

Topic_names = {   }
#Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic'].map(Topic_names)

In [ ]:
df_clean

## Supervised model to predict any new complaints to the relevant Topics.

You have now build the model to create the topics for each complaints.Now in the below section you will use them to classify any new complaints.

Since you will be using supervised learning technique we have to convert the topic names to numbers(numpy arrays only understand numbers)

In [ ]:
#Create the dictionary again of Topic names and Topics

Topic_names = {   }
#Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic'].map(Topic_names)

In [ ]:
df_clean

In [ ]:
#Keep the columns"complaint_what_happened" & "Topic" only in the new dataframe --> training_data
training_data=

In [ ]:
training_data

####Apply the supervised models on the training data created. In this process, you have to do the following:
* Create the vector counts using Count Vectoriser
* Transform the word vecotr to tf-idf
* Create the train & test data using the train_test_split on the tf-idf & topics


In [15]:

#Write your code to get the Vector count


#Write your code here to transform the word vector to tf-idf

You have to try atleast 3 models on the train & test data from these options:
* Logistic regression
* Decision Tree
* Random Forest
* Naive Bayes (optional)

**Using the required evaluation metrics judge the tried models and select the ones performing the best**

In [16]:
# Write your code here to build any 3 models and evaluate them using the required metrics



